<a href="https://colab.research.google.com/github/pedrogengo/DLforNLP/blob/main/Pedro_Gengo_Aula_10_Exerc%C3%ADcio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Usar as secções como guia.

Nome: Pedro Gabriel Gengo Lourenço

## Instruções

Neste colab iremos treinar um modelo T5 para traduzir de inglês para português. Iremos treiná-lo com o data Paracrawl.

- Usaremos o dataset Paracrawl Inglês-Português. Truncamos o dataset de treino para apenas 100k pares para deixar o treinamento mais rápido. Quem quiser pode treinar com mais amostras. Se demorar muito para treinar, truncar o dataset ainda mais.

- Usaremos o BLEU como métrica. Usaremos o SacreBLEU pois sempre faz o mesmo pré-processamento (tokenização, lowercase). Não usaremos torchnlp.metrics.bleu, torchtext.data.metrics.bleu_score, etc. SacreBLEU é lento: usar poucas amostras de validação (ex: 5k)


Usaremos o modelo PTT5 disponível no model hub da HuggingFace:

https://huggingface.co/unicamp-dl/ptt5-small-portuguese-vocab

Este é  um T5 pré-treinado em textos em português e com tokenizador em português. 


In [ ]:
# Configurações gerais
model_name = "unicamp-dl/ptt5-small-portuguese-vocab"
batch_size = 64
accumulate_grad_batches = 2
source_max_length = 128
target_max_length = 128
learning_rate = 1e-3

In [ ]:
! pip install sacrebleu
! pip install pytorch-lightning
! pip install transformers
! pip install sentencepiece

In [ ]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.
import gzip
import nvidia_smi
import os
import pytorch_lightning as pl
import random
import sacrebleu
import torch
import torch.nn.functional as F

from google.colab import drive

from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from typing import Dict
from typing import List
from typing import Tuple

In [ ]:
# Important: Fix seeds so we can replicate results
seed = 123
random.seed(seed)
# np.random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

DICA para modelos reais: Um modelo otimizado deve manter o uso de GPU próximo a 100% durante o treino.
Vamos utilizar a bilioteca abaixo para monitorar isso. Note que no modelo simples utilizado aqui o uso não vai chegar a 100%.

In [ ]:
print(f"Pytorch Lightning Version: {pl.__version__}")
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
print(f"Device name: {nvidia_smi.nvmlDeviceGetName(handle)}")

def gpu_usage():
    global handle
    return str(nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu) + '%'

Pytorch Lightning Version: 1.4.9
Device name: b'Tesla K80'


Iremos salvar os checkpoints (pesos do modelo) no google drive, para que possamos continuar o treino de onde paramos.

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
! wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_train.tsv.gz
! wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_test.tsv.gz

File ‘paracrawl_enpt_train.tsv.gz’ already there; not retrieving.

File ‘paracrawl_enpt_test.tsv.gz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (100k pares) e val (5k pares) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
def load_text_pairs(path):
    text_pairs = []
    for line in gzip.open(path, mode='rt'):
        text_pairs.append(line.strip().split('\t'))
    return text_pairs

x_train = load_text_pairs('paracrawl_enpt_train.tsv.gz')
x_test = load_text_pairs('paracrawl_enpt_test.tsv.gz')

# Embaralhamos o treino para depois fazermos a divisão treino/val.
random.shuffle(x_train)

# Truncamos o dataset para 100k pares de treino e 5k pares de validação.
x_val = x_train[100000:105000]
x_train = x_train[:100000]

for set_name, x in [('treino', x_train), ('validação', x_val), ('test', x_test)]:
    print(f'\n{len(x)} amostras de {set_name}')
    print(f'3 primeiras amostras {set_name}:')
    for i, (source, target) in enumerate(x[:3]):
        print(f'{i}: source: {source}\n   target: {target}')


100000 amostras de treino
3 primeiras amostras treino:
0: source: More Croatian words and phrases
   target: Mais palavras e frases em croata
1: source: Jerseys and pullovers, containing at least 50Â % by weight of wool and weighing 600Â g or more per article 6110 11 10 (PCE)
   target: Camisolas e pulôveres, com pelo menos 50 %, em peso, de lã e pesando 600g ou mais por unidade 6110 11 10 (PCE)
2: source: Atex Colombia SAS makes available its lead product, 100% natural liquid latex, excellent quality and price. ... Welding manizales caldas Colombia a DuckDuckGo
   target: Atex Colômbia SAS torna principal produto está disponível, látex líquido 100% natural, excelente qualidade e preço. ...

5000 amostras de validação
3 primeiras amostras validação:
0: source: «You have hidden these things from the wise and the learned you have revealed them to the childlike»
   target: «Escondeste estas coisas aos sábios e entendidos e as revelaste aos pequenos»
1: source: Repair of computers, applic

Criando Dataset


In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [ ]:
x_train[1]

['Jerseys and pullovers, containing at least 50Â % by weight of wool and weighing 600Â g or more per article 6110 11 10 (PCE)',
 'Camisolas e pulôveres, com pelo menos 50 %, em peso, de lã e pesando 600g ou mais por unidade 6110 11 10 (PCE)']

In [ ]:
tokenizer(x_train[1][0], return_tensors="pt").input_ids[0]

tensor([11665,     6,   599,  7772,   124,  3712,     3,   432,   193,   479,
            7,    71,  1373,  9761,  1094, 23295, 13279,  8037,    31,  1528,
         4276,   224,    31,  4331,  1388,   599,  6168,  1954,  2859,   479,
         7226, 23295,  2475,     9,    33,    31,  5010,   848,     7,  6962,
         3725,  8731,  1460,   334,   230,    24,   388,  5209,    36,     1])

In [ ]:
class MyDataset(Dataset):
    def __init__(self, text_pairs: List[Tuple[str]], tokenizer, prefix_task,
                 source_max_length: int = 32, target_max_length: int = 32):
        self.tokenizer = tokenizer
        self.text_pairs = text_pairs
        self.prefix_task = prefix_task
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length
        
    def __len__(self):
        return len(self.text_pairs)
    
    def __getitem__(self, idx):
        source, target = self.text_pairs[idx]
        full_source = self.prefix_task + source

        # Tokenizacao do source
        tokenized_source = tokenizer(full_source, padding='max_length',
                                     max_length=self.source_max_length,
                                     truncation=True,
                                     return_tensors="pt")
        source_token_ids, source_mask = tokenized_source.input_ids[0], tokenized_source.attention_mask[0]

        # Tokenizacao do target
        tokenized_target = tokenizer(target, padding='max_length',
                                     max_length=self.target_max_length,
                                     truncation=True,
                                     return_tensors="pt")
        target_token_ids, target_mask = tokenized_target.input_ids[0], tokenized_target.attention_mask[0]

        
        return (source_token_ids, source_mask,
                target_token_ids, target_mask,
                source, target)

## Testando o DataLoader

In [ ]:
text_pairs = [('we like pizza', 'eu gosto de pizza')]
dataset_debug = MyDataset(
    text_pairs=text_pairs,
    tokenizer=tokenizer,
    prefix_task='translate English to Portuguese: ',
    source_max_length=source_max_length,
    target_max_length=target_max_length)

dataloader_debug = DataLoader(dataset_debug, batch_size=10, shuffle=True, 
                              num_workers=0)

source_token_ids, source_mask, target_token_ids, target_mask, _, _ = next(iter(dataloader_debug))
print('source_token_ids:\n', source_token_ids)
print('source_mask:\n', source_mask)
print('target_token_ids:\n', target_token_ids)
print('target_mask:\n', target_mask)

print('source_token_ids.shape:', source_token_ids.shape)
print('source_mask.shape:', source_mask.shape)
print('target_token_ids.shape:', target_token_ids.shape)
print('target_mask.shape:', target_mask.shape)

source_token_ids:
 tensor([[ 1246,   104,   146, 20739,   934, 15374,  1066,    32,    46,    31,
          1528,  1079,   634,  1241,  7531,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,    

## Criando DataLoaders de Treino/Val/Test

In [ ]:
prefix_task='translate English to Portuguese: '

dataset_train = MyDataset(text_pairs=x_train,
                          tokenizer=tokenizer,
                          prefix_task=prefix_task,
                          source_max_length=source_max_length,
                          target_max_length=target_max_length)

dataset_val = MyDataset(text_pairs=x_val,
                        tokenizer=tokenizer,
                        prefix_task=prefix_task,
                        source_max_length=source_max_length,
                        target_max_length=target_max_length)

dataset_test = MyDataset(text_pairs=x_test,
                         tokenizer=tokenizer,
                         prefix_task=prefix_task,
                         source_max_length=source_max_length,
                         target_max_length=target_max_length)

train_dataloader = DataLoader(dataset_train, batch_size=batch_size,
                              shuffle=True, num_workers=0)

val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, 
                            num_workers=0)

test_dataloader = DataLoader(dataset_test, batch_size=batch_size,
                             shuffle=False, num_workers=0)

## Criando o T5 com Pytorch Lightning

In [ ]:
class T5Finetuner(pl.LightningModule):

    def __init__(self, tokenizer, train_dataloader, val_dataloader,
                 test_dataloader, learning_rate, target_max_length=32):
        super(T5Finetuner, self).__init__()
        
        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader
        self._test_dataloader = test_dataloader

        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        
        self.tokenizer = tokenizer
        self.learning_rate = learning_rate
        self.target_max_length = target_max_length

        self.bleu = sacrebleu.BLEU()

    def forward(self, source_token_ids, source_mask, target_token_ids=None,
                target_mask=None):
      
        if self.training:
            # TODO: calcular a loss dado os target_token_ids
            target_token_ids[target_token_ids==self.tokenizer.pad_token_id] = -100
            loss = self.model(input_ids=source_token_ids, attention_mask=source_mask, labels=target_token_ids).loss
            return loss
        else:
            # TODO: gerar predicted_token ids
            generated_ids = self.model.generate(source_token_ids, attention_mask=source_mask, max_length=self.target_max_length)
            return generated_ids

    def training_step(self, batch, batch_nb):
        source_token_ids, source_mask, target_token_ids, target_mask, _, _ = batch
         
        # fwd
        loss = self(source_token_ids, source_mask, target_token_ids, target_mask)

        # logs
        tensorboard_logs = {'train_loss': loss}
        progress_bar = {'gpu_usage': gpu_usage()}
        return {'loss': loss, 'log': tensorboard_logs,
                'progress_bar': progress_bar}

    def validation_step(self, batch, batch_nb):
        # TODO: Adicionar sacrebleu.
        source_token_ids, source_mask, target_token_ids, target_mask, source, targets = batch
        predicted_ids = self.forward(source_token_ids, source_mask) #gera ids
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

        avg_bleu = self.bleu.corpus_score(predicted_texts, [targets]).score

        # TODO: Imprimir source, target e predicted para avaliação qualitativa.
        print('*'*20)
        print(f"Source: {source[0]}")
        print(f"Target: {targets[0]}")
        print(f"Predicted: {predicted_texts[0]}")
        print('*'*20)
        return {'val_bleu': avg_bleu}

    def test_step(self, batch, batch_nb):
        # TODO: Adicionar sacrebleu.
        source_token_ids, source_mask, target_token_ids, target_mask, source, targets = batch
        predicted_ids = self.forward(source_token_ids, source_mask) #gera ids
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

        avg_bleu = self.bleu.corpus_score(predicted_texts, [targets]).score

        # TODO: Imprimir source, target e predicted para avaliação qualitativa.
        print('*'*20)
        print(f"Source: {source[0]}")
        print(f"Target: {targets[0]}")
        print(f"Predicted: {predicted_texts[0]}")
        print('*'*20)
        return {'test_bleu': avg_bleu}

    def validation_epoch_end(self, outputs):
        avg_bleu = sum([x['val_bleu'] for x in outputs]) / len(outputs)

        self.log("avg_val_bleu", avg_bleu, prog_bar=True)

    def test_epoch_end(self, outputs):
        avg_bleu = sum([x['test_bleu'] for x in outputs]) / len(outputs)

        self.log("avg_test_bleu", avg_bleu, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.learning_rate, eps=1e-08)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=1.0)  # This is the same as no LR decay.
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'avg_val_bleu'}

    def train_dataloader(self):
        return self._train_dataloader

    def val_dataloader(self):
        return self._val_dataloader

    def test_dataloader(self):
        return self._test_dataloader

In [ ]:
model = T5Finetuner(tokenizer=tokenizer,
                    train_dataloader=train_dataloader,
                    val_dataloader=val_dataloader,
                    test_dataloader=test_dataloader,
                    learning_rate=learning_rate, 
                    target_max_length=target_max_length)

## Testando rapidamente o modelo em treino, validação e teste com um batch

In [ ]:
trainer = pl.Trainer(gpus=1,
                     precision=16, 
                     checkpoint_callback=False,  # Disable checkpoint saving.
                     fast_dev_run=True)
trainer.fit(model)
trainer.test(model)
del model  # Para não ter estouro de mémoria da GPU

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging

Training: -1it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:406: LightningDeprecationWarning: One of the returned values {'progress_bar', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", U

Validating: 0it [00:00, ?it/s]

********************
Source: «You have hidden these things from the wise and the learned you have revealed them to the childlike»
Target: «Escondeste estas coisas aos sábios e entendidos e as revelaste aos pequenos»
Predicted: translate English to Portuguese: «You have hidden these things from the wise and the learned you have revealed them to the childlike»
********************


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

********************
Source: In this way, the civil life of a nation matures, making it possible for all citizens to enjoy the fruits of genuine tolerance and mutual respect.
Target: Deste modo, a vida civil de uma nação amadurece, fazendo com que todos os cidadãos gozem dos frutos da tolerância genuína e do respeito mútuo.
Predicted: translate English to Portuguese: In this way, the civil life of a nation matures, making it possible for all citizens to enjoy the fruits of genuine tolerance and mutual respect.
********************
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_bleu': 4.8095526695251465}
--------------------------------------------------------------------------------


## Overfit em algumas amostras

Antes de treinar o modelo no dataset todo, faremos overfit do 
modelo em poucas de treino para verificar se loss vai para próximo de 0. Isso serve para depurar se a implementação do modelo está correta.

Podemos também medir se a acurácia neste minibatch chega perto de 100%. Isso serve para depurar se nossa função que mede a acurácia está correta.

Nota: se treinarmos por muitas épocas (ex: 500) é possivel que a loss vá para zero mesmo com bugs na implementação. O ideal é que a loss chege próxima a zero antes de 100 épocas.

In [ ]:
trainer = pl.Trainer(gpus=1,
                     precision=16,
                     max_epochs=30,
                     check_val_every_n_epoch=10,
                     checkpoint_callback=False,  # Disable checkpoint saving
                     overfit_batches=0.005)

# Dataset usando apenas um batch de amostras de treino.
dataset_debug = MyDataset(text_pairs=x_train,
                          tokenizer=tokenizer,
                          prefix_task=prefix_task,
                          source_max_length=source_max_length,
                          target_max_length=target_max_length)

debug_dataloader = DataLoader(dataset_debug, batch_size=batch_size,
                              shuffle=False, num_workers=0)

model = T5Finetuner(tokenizer=tokenizer,
                    train_dataloader=debug_dataloader,
                    val_dataloader=debug_dataloader,
                    test_dataloader=None,
                    learning_rate=learning_rate, 
                    target_max_length=target_max_length)

trainer.fit(model)
del model  # Para não ter estouro de mémoria da GPU

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

********************
Source: More Croatian words and phrases
Target: Mais palavras e frases em croata
Predicted: translate English to Portuguese: More Croatian words and phrases
********************
********************
Source: January 2015 – December 2017
Target: Janeiro de 2015 – Dezembro de 2017
Predicted: translate English to Portuguese: January 2015 – December 2017
********************


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

...

********************
Source: More Croatian words and phrases
Target: Mais palavras e frases em croata
Predicted: More Croatian tem várias opções e suas opções de leitura e leitura
********************
********************
Source: January 2015 – December 2017
Target: Janeiro de 2015 – Dezembro de 2017
Predicted: January 2015 – December 2017
********************
********************
Source: There might also be a course in cultural competence that consists of answering questions on multiple-choice tests.
Target: Também pode haver um curso de competência cultural que consiste em responder a perguntas sobre os testes de múltipla escolha.
Predicted: O que é a cultura cultural é a única que pode ser a melhor forma de se tornar uma cultura cultural.
********************
********************
Source: What? You are what?
Target: Vocês estão o quê?
Predicted: Você não pode reclamar?
********************
********************
Source: In the fiscal year to 31 March 2013, Illovo Sugar posted a 41 perc

Validating: 0it [00:00, ?it/s]

********************
Source: More Croatian words and phrases
Target: Mais palavras e frases em croata
Predicted: More Croatian conta histórias e suas técnicas e suas técnicas em suas técnicas e suas técnicas
********************
********************
Source: January 2015 – December 2017
Target: Janeiro de 2015 – Dezembro de 2017
Predicted: Janeiro de 2015 – Dezembro de 2017
********************
********************
Source: There might also be a course in cultural competence that consists of answering questions on multiple-choice tests.
Target: Também pode haver um curso de competência cultural que consiste em responder a perguntas sobre os testes de múltipla escolha.
Predicted: É importante destacar que há uma tendência a pensar em questões de múltipla escolha entre as culturas culturais.
********************
********************
Source: What? You are what?
Target: Vocês estão o quê?
Predicted: Vocês estão a falar o que acham dele?
********************
********************
Source: In th

Validating: 0it [00:00, ?it/s]

********************
Source: More Croatian words and phrases
Target: Mais palavras e frases em croata
Predicted: Mais palavras e frases em grego e suas frases
********************
********************
Source: January 2015 – December 2017
Target: Janeiro de 2015 – Dezembro de 2017
Predicted: Janeiro de 2015 – Dezembro de 2017
********************
********************
Source: There might also be a course in cultural competence that consists of answering questions on multiple-choice tests.
Target: Também pode haver um curso de competência cultural que consiste em responder a perguntas sobre os testes de múltipla escolha.
Predicted: É importante participar de uma série de eventos em grupo que tratam de questões de múltipla escolha.
********************
********************
Source: What? You are what?
Target: Vocês estão o quê?
Predicted: Vocês estão o quê?
********************
********************
Source: In the fiscal year to 31 March 2013, Illovo Sugar posted a 41 percent increase in ope

## Treinamento e Validação no dataset todo

In [ ]:
max_epochs = 5

checkpoint_path = 'drive/My Drive/checkpoints/epoch=10.ckpt'
checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
os.makedirs(checkpoint_dir, exist_ok=True)
print(f'Files in {checkpoint_dir}: {os.listdir(checkpoint_dir)}')
print(f'Saving checkpoints to {checkpoint_dir}')
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir,
                                      save_top_k=-1)  # Keeps all checkpoints.

resume_from_checkpoint = None
if os.path.exists(checkpoint_path):
    print(f'Restoring checkpoint: {checkpoint_path}')
    resume_from_checkpoint = checkpoint_path

trainer = pl.Trainer(gpus=1,
                     precision=16,
                     max_epochs=max_epochs,
                     check_val_every_n_epoch=1,
                     accumulate_grad_batches=accumulate_grad_batches,
                     callbacks=[checkpoint_callback],
                     progress_bar_refresh_rate=50,
                     resume_from_checkpoint=resume_from_checkpoint)

model = T5Finetuner(tokenizer=tokenizer,
                    train_dataloader=train_dataloader,
                    val_dataloader=val_dataloader,
                    test_dataloader=test_dataloader,
                    learning_rate=learning_rate, 
                    target_max_length=target_max_length)

trainer.fit(model)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files in /content/drive/My Drive/checkpoints: []
Saving checkpoints to /content/drive/My Drive/checkpoints


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

KeyboardInterrupt: ignored

## Após treinado, avaliamos o modelo no dataset de teste

É importante que essa avaliação seja feita poucas vezes para evitar "overfit manual" no dataset de teste.

In [ ]:
trainer.test(model)